In [14]:
import cv2
import numpy as nps
from keras.models import model_from_json
from sklearn.metrics import accuracy_score

In [15]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [16]:
emotion_dict[0]

'Angry'

In [17]:
emotion_dict

{0: 'Angry',
 1: 'Disgusted',
 2: 'Fearful',
 3: 'Happy',
 4: 'Neutral',
 5: 'Sad',
 6: 'Surprised'}

In [18]:
# load json and create model
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [19]:
# load weights into new model
emotion_model.load_weights("model/emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [20]:
import matplotlib.pyplot as plt

In [21]:
# start the webcam feed
cap = cv2.VideoCapture(0)


# pass here video path+
# Video download  : https://www.pexels.com/video/three-girls-laughing-5273028/
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\A.mp4")
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\B.mp4")
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\C.mp4")
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\D.mp4")
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\E.mp4")
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\F.mp4")
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\G.mp4")
# cap = cv2.VideoCapture("D:\Project\HED\Emotion_detection_with_CNN-main\Video\pexels-polina-tankilevitch-3875281-1920x1080-25fps.mp4")


Detect the faces, crop it and then passes it to the emotion detector model and then emotion detection model will detect the emotion from that

In [22]:
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w] 
        # Cropping the grayframe
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [23]:
emotion_model

In [24]:
import matplotlib.pyplot as plt

In [25]:
def plot_model_history(model_history):
    """
    Plot Accuracy and Loss curves given the model_history
    """
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('plot.png')
    plt.show()

In [26]:
#  plot_model_history(emotion_model)